# Parcialito Cálculo Numérico
**FCEyN - UBA**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## **Ejercicio 1: Método de Gauss-Seidel**
Resolver Ax = b con A = [[4,-1,0],[2,7,1],[0,-3,5]], b = [15,20,10]

In [ ]:
def gauss_seidel(A, b, x0, tol, max_iter):
    n = len(A)
    x = x0.copy()
    
    for k in range(max_iter):
        x_ant = x.copy()
        
        for i in range(n):
            suma = 0
            for j in range(n):
                if i != j:
                    suma += A[i,j] * x[j]
            x[i] = (b[i] - suma) / A[i,i]
        
        if np.linalg.norm(x - x_ant) < tol:
            return x, k+1
    
    return x, max_iter

In [ ]:
A = np.array([[4,-1,0],[2,7,1],[0,-3,5]], dtype=float)
b = np.array([15,20,10], dtype=float)
x0 = np.array([0,0,0], dtype=float)
tol = 1e-6
max_iter = 100

sol, iter = gauss_seidel(A, b, x0, tol, max_iter)
print(f"Solución: {sol}")
print(f"Iteraciones: {iter}")
print(f"Verificación Ax-b: {np.linalg.norm(A@sol - b)}")

## **Ejercicio 2: Método de Euler - EDO**
Resolver y' = 2t - y + 1, y(0) = 0 en [0,2] con h=0.1

In [ ]:
def euler(f, t0, tf, y0, h):
    t = np.arange(t0, tf + h, h)
    y = np.zeros(len(t))
    y[0] = y0
    
    for i in range(1, len(t)):
        y[i] = y[i-1] + h * f(t[i-1], y[i-1])
    
    return t, y

In [ ]:
def f(t, y):
    return 2*t - y + 1

t, y = euler(f, 0, 2, 0, 0.1)

plt.plot(t, y)
plt.xlabel('t')
plt.ylabel('y(t)')
plt.title('Solución EDO')
plt.grid(True)
plt.show()

idx = np.where(np.abs(t - 1.0) < 0.01)[0][0]
print(f"y(1) = {y[idx]:.4f}")

## **Ejercicio 3: Estimación Norma de Matriz**
Estimar norma-2 de A = [[3,1,-1],[1,4,2],[-1,2,5]]

In [ ]:
def estimar_norma(A, n_iter):
    s = np.zeros(n_iter)
    s[0] = 0
    
    for k in range(1, n_iter):
        x = np.random.randn(A.shape[0])
        x = x / np.linalg.norm(x)
        
        Ax = A @ x
        norma_Ax = np.linalg.norm(Ax)
        
        s[k] = max(s[k-1], norma_Ax)
    
    return s

In [ ]:
A = np.array([[3,1,-1],[1,4,2],[-1,2,5]], dtype=float)

s = estimar_norma(A, 50)

plt.plot(s)
plt.axhline(y=np.linalg.norm(A, 2), color='r', linestyle='--', label='Norma exacta')
plt.xlabel('Iteración')
plt.ylabel('Estimación')
plt.legend()
plt.show()

print(f"Norma estimada: {s[-1]:.6f}")
print(f"Norma exacta: {np.linalg.norm(A, 2):.6f}")

## **Ejercicio 4: Diferencias Finitas**
Resolver -u'' + 2u = x² en [0,1] con u(0)=1, u(1)=3

In [ ]:
def dif_finitas(N, alpha, u0, u1, f_func):
    h = 1.0 / (N + 1)
    x = np.linspace(0, 1, N + 2)
    
    # Matriz A
    A = np.zeros((N, N))
    for i in range(N):
        A[i, i] = -2/h**2 + alpha
        if i > 0:
            A[i, i-1] = 1/h**2
        if i < N-1:
            A[i, i+1] = 1/h**2
    
    # Vector b
    b = np.zeros(N)
    for i in range(N):
        b[i] = f_func(x[i+1])
    
    # Condiciones frontera
    b[0] -= u0 / h**2
    b[-1] -= u1 / h**2
    
    u_int = np.linalg.solve(A, b)
    u = np.concatenate([[u0], u_int, [u1]])
    
    return x, u

In [ ]:
def f(x):
    return x**2

x, u = dif_finitas(10, 2.0, 1, 3, f)

plt.plot(x, u, 'b-o')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('Diferencias Finitas')
plt.grid(True)
plt.show()

idx = len(x)//2
print(f"u(0.5) = {u[idx]:.4f}")

## **Ejercicio 5: Sistema de EDOs**
y1' = y2, y2' = -y1 - 0.1*y2 con y1(0)=1, y2(0)=0

In [ ]:
def euler_sistema(F, t0, tf, y0, h):
    t = np.arange(t0, tf + h, h)
    y = np.zeros((len(y0), len(t)))
    y[:, 0] = y0
    
    for i in range(1, len(t)):
        y[:, i] = y[:, i-1] + h * F(t[i-1], y[:, i-1])
    
    return t, y

In [ ]:
def sistema(t, y):
    y1, y2 = y[0], y[1]
    dy1 = y2
    dy2 = -y1 - 0.1*y2
    return np.array([dy1, dy2])

t, Y = euler_sistema(sistema, 0, 10, np.array([1, 0]), 0.05)

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(t, Y[0, :], label='y1(t)')
plt.plot(t, Y[1, :], label='y2(t)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(Y[0, :], Y[1, :])
plt.xlabel('y1')
plt.ylabel('y2')
plt.title('Retrato de fase')
plt.grid(True)
plt.show()

## **Ejercicio 6: Eliminación Gaussiana**
Resolver Ax = b con A = [[2,1,-1],[4,5,3],[2,1,4]], b = [8,9,11]

In [ ]:
def gauss_elim(A, b):
    n = len(A)
    A = A.copy().astype(float)
    b = b.copy().astype(float)
    
    # Eliminación
    for k in range(n-1):
        for i in range(k+1, n):
            factor = A[i, k] / A[k, k]
            for j in range(k, n):
                A[i, j] -= factor * A[k, j]
            b[i] -= factor * b[k]
    
    # Sustitución hacia atrás
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = b[i]
        for j in range(i+1, n):
            x[i] -= A[i, j] * x[j]
        x[i] /= A[i, i]
    
    return A, x

In [ ]:
A = np.array([[2,1,-1],[4,5,3],[2,1,4]])
b = np.array([8,9,11])

A_esc, x = gauss_elim(A, b)

print("Matriz escalonada:")
print(A_esc)
print(f"\nSolución: {x}")
print(f"Verificación: {np.allclose(A @ x, b)}")

## **Ejercicio 7: EDO con parámetro**
y' = (y-3)(sin²(t)-0.3) para diferentes condiciones iniciales

In [ ]:
def f_param(t, y):
    return (y - 3) * (np.sin(t)**2 - 0.3)

plt.figure(figsize=(10, 6))
for k in [0, 1, 2, 3, 4, 5, 6]:
    t, y = euler(f_param, 0, 5, k, 0.01)
    plt.plot(t, y, label=f'y(0)={k}')

plt.axhline(y=3, color='black', linestyle='--', alpha=0.7, label='y=3')
plt.xlim(0, 5)
plt.ylim(-2, 8)
plt.legend()
plt.grid(True)
plt.show()

## **Ejercicio 8: Método de Jacobi**
Resolver Ax = b con A = [[10,-1,2],[-1,11,-1],[2,-1,10]], b = [6,25,-11]

In [ ]:
def jacobi(A, b, x0, tol, max_iter):
    n = len(A)
    x = x0.copy()
    
    for k in range(max_iter):
        x_nuevo = np.zeros(n)
        
        for i in range(n):
            suma = 0
            for j in range(n):
                if i != j:
                    suma += A[i, j] * x[j]
            x_nuevo[i] = (b[i] - suma) / A[i, i]
        
        if np.linalg.norm(x_nuevo - x) < tol:
            return x_nuevo, k+1, True
        
        x = x_nuevo.copy()
    
    return x, max_iter, False

In [ ]:
A = np.array([[10,-1,2],[-1,11,-1],[2,-1,10]], dtype=float)
b = np.array([6,25,-11], dtype=float)
x0 = np.array([0,0,0], dtype=float)

sol, iter, convergio = jacobi(A, b, x0, 1e-8, 1000)

if convergio:
    print(f"Convergió en {iter} iteraciones")
    print(f"Solución: {sol}")
else:
    print("No convergió")
    print(f"Última aproximación: {sol}")

## **Ejercicio 9: Matriz Inversa**
Calcular A⁻¹ para A = [[4,7],[2,6]]

In [ ]:
def matriz_inv(A):
    n = len(A)
    A_aug = np.hstack([A.copy().astype(float), np.eye(n)])
    
    # Eliminación Gauss-Jordan
    for i in range(n):
        # Pivoteo
        A_aug[i] = A_aug[i] / A_aug[i, i]
        
        for j in range(n):
            if i != j:
                A_aug[j] = A_aug[j] - A_aug[j, i] * A_aug[i]
    
    return A_aug[:, n:]

In [ ]:
A = np.array([[4,7],[2,6]])

det_A = np.linalg.det(A)
print(f"Determinante: {det_A}")

if abs(det_A) > 1e-10:
    A_inv = matriz_inv(A)
    print("\nA inversa:")
    print(A_inv)
    
    print("\nVerificación A*A_inv:")
    print(A @ A_inv)
    
    print("\nComparación con numpy:")
    print(np.linalg.inv(A))
else:
    print("Matriz no invertible")

## **Ejercicio 10: Convergencia Norma**
Para A = [[1,2,0],[0,3,1],[1,0,2]] estudiar convergencia

In [ ]:
A = np.array([[1,2,0],[0,3,1],[1,0,2]], dtype=float)

s = estimar_norma(A, 100)
norma_exacta = np.linalg.norm(A, 2)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(s)
plt.axhline(y=norma_exacta, color='r', linestyle='--', label='Norma exacta')
plt.xlabel('k')
plt.ylabel('s_k')
plt.title('Convergencia completa')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(s[:20], 'o-')
plt.axhline(y=norma_exacta, color='r', linestyle='--', label='Norma exacta')
plt.xlabel('k')
plt.ylabel('s_k')
plt.title('Primeras 20 iteraciones')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Valor final: {s[-1]:.6f}")
print(f"Norma exacta: {norma_exacta:.6f}")
print(f"Es monótona: {all(s[i] <= s[i+1] for i in range(len(s)-1))}")